In [93]:
import subprocess
import requests
requests.packages.urllib3.disable_warnings() 

host = '10.0.118.224'

ssh = subprocess.Popen(
    ["ssh", 
     '-i {}'.format('/Users/weijin.wang/ws/sdmain/deployment/ssh_keys/ubuntu.pem'),
     'ubuntu@{}'.format(host), 
     '/opt/rubrik/src/scripts/dev/get_local_spray_token.py --username spark$'],
    shell=False,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE)
result = ssh.stdout.readlines()
if not result:
    error = ssh.stderr.readlines()
    print(error)
else:
    print(ssh.stdout, result)

auth_token = result[0].strip()


In [97]:
#print('token {}'.format(auth_token))

rsp = requests.get(
    'https://10.0.118.224/api/internal/app_blueprint', 
    headers={"Authorization": "Bearer {}".format(auth_token)},
    verify=False)

blueprints = []
if rsp.status_code == 200:
    content_json = rsp.json()
    #print(content_json)
    for bp in content_json['data']:
        #print(bp)
        if not bp['isRelic'] and bp['primaryClusterId'] == '4f66ee92-6e7e-4c2c-a3db-a0b128eab32a':
            print(bp['id'], bp['name'], bp['primaryClusterId'])
            blueprints.append(bp['id'])

print('Found {} blueprints'.format(len(blueprints)))



Found 0 blueprints


In [51]:
# delete
if len(blueprints) != 0:
    for bp_id in blueprints:
        rsp = requests.delete(
            'https://10.0.118.224/api/internal/polaris/app_blueprint/' + bp_id, 
            headers={"Authorization": "Bearer {}".format(auth_token)},
            verify=False)
        print(rsp)



<Response [204]>


<Response [204]>


<Response [204]>


<Response [204]>


<Response [204]>


<Response [204]>


<Response [204]>
